In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Tue Feb  8 10:20:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import json
import os
from collections import Counter
import random
import numpy as np
!pip install transformers
import torch
import torch.nn as nn
from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer
)
import transformers
from torchsummary import summary
from tqdm import tqdm
from tqdm import tqdm, trange
from transformers import  get_linear_schedule_with_warmup


from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

## **DATASET**

In [ ]:
import re
def simple_text_clean(x):
    # first we lowercase everything
    x = x.lower()
    # remove unicode characters
    x = x.encode('ascii', 'ignore').decode()
    #remove websites
    x = re.sub(r'[\S]+\.(net|com|org|info|edu|gov|uk|de|ca|jp|fr|au|us|ru|ch|it|nel|se|no|es|mil)[\S]*\s?','',x)
    x = re.sub(r'[^a-zA-z.,!?/;\"\'\s]','',x) #removes special characters except some & remove nums 0-9
    return x

In [ ]:
import pandas as pd
traindf=pd.read_csv('/content/drive/MyDrive/MAMI_DATASET/training.csv',sep='\t')
traindf.columns = [*traindf.columns[:-1], 'text']
traindf['text'] =traindf['text'].apply(str).apply(lambda x: simple_text_clean(x))
traindf

,file_name,misogynous,shaming,stereotype,objectification,violence,text
0,1.jpg,0,0,0,0,0,milk milk.zip
1,10.jpg,1,0,0,0,1,"roses are red, violets are blue if you don't s..."
2,1000.jpg,0,0,0,0,0,breaking news russia releases photo of donald ...
3,10000.jpg,0,0,0,0,0,man seeking woman ignad o
4,10006.jpg,0,0,0,0,0,me explaining the deep lore of. j.r.r. tolkein...
...,...,...,...,...,...,...,...
9995,15002.jpg,0,0,0,0,0,waiting for the end of the covid
9996,15003.jpg,0,0,0,0,0,smart women are around
9997,15004.jpg,0,0,0,0,0,good girls are behind the corner
9998,15005.jpg,0,0,0,0,0,cooking for my wife


In [ ]:
evaldf=pd.read_csv('/content/drive/MyDrive/MAMI_DATASET/test/trial.csv',sep='\t')
evaldf.columns = [*evaldf.columns[:-1], 'text']
evaldf['text'] =evaldf['text'].apply(str).apply(lambda x: simple_text_clean(x))
evaldf

,file_name,misogynous,shaming,stereotype,objectification,violence,text
0,28.jpg,0,0,0,0,0,"not now, dad. we should burn jon snow. stop it..."
1,30.jpg,0,0,0,0,0,there may have been a mixcommunication with th...
2,33.jpg,0,0,0,0,0,i shouldn't have sold my boat
3,58.jpg,1,0,0,0,1,"bitches be like, it was my fault i made him mad"
4,89.jpg,0,0,0,0,0,find a picture of girls together on fb make a...
...,...,...,...,...,...,...,...
95,1380.jpg,1,0,1,0,0,rape culture. it's what every oxymoronic femi...
96,1381.jpg,0,0,0,0,0,"walking, running, telereporting, not going to ..."
97,1384.jpg,1,0,0,0,1,taking the time to get her pussy wet. always p...
98,1408.jpg,0,0,0,0,0,what men play with vs what women play with


In [ ]:
class ViTDataset(Dataset):
    def __init__(self, dataset,path, tokenizer, transforms, max_seq_length):
        self.image_ids = dataset.file_name
        self.image_labels = dataset.misogynous
        self.img_dir = path
        self.tokenizer = tokenizer
        self.transforms = transforms
        self.max_seq_length = max_seq_length
        self.text = dataset.text

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self,idx):
        sentence = torch.LongTensor(self.tokenizer.encode(self.text[idx], add_special_tokens=True))
        # image path
        img_path =os.path.join(self.img_dir,self.image_ids[idx])
        image = Image.open(img_path).convert("RGB")
        # transform image
        if self.transforms:
                image = self.transforms(image)
        label = self.image_labels[idx]
        label = torch.from_numpy(np.array(label, dtype='int32'))

        res ={
            "sentence":sentence,
            "image": image,
            "label": label}
        
        return res

In [ ]:
def collate_fn(batch):
    lens = [len(row["sentence"]) for row in batch]
    bsz, max_seq_len = len(batch), max(lens)

    mask_tensor = torch.zeros(bsz, max_seq_len, dtype=torch.long)
    text_tensor = torch.zeros(bsz, max_seq_len, dtype=torch.long)

    for i_batch, (input_row, length) in enumerate(zip(batch, lens)):
        text_tensor[i_batch, :length] = input_row["sentence"]
        mask_tensor[i_batch, :length] = 1

    img_tensor = torch.stack([row["image"] for row in batch])
    tgt_tensor = torch.stack([row["label"] for row in batch])
    
    return text_tensor, mask_tensor, img_tensor,tgt_tensor

In [ ]:
!pip install torchvision
!pip install scipy==1.2.0
import torchvision.transforms as transforms
import torchvision

In [ ]:
def get_image_transforms():
    return transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.46777044, 0.44531429, 0.40661017],
                std=[0.12221994, 0.12145835, 0.14380469],
            ),
        ]
    )

In [ ]:
from transformers import ViTFeatureExtractor, ViTModel
from transformers import BertModel
bertmodel = BertModel.from_pretrained("bert-base-uncased",output_hidden_states=True)
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")
vitmodel = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True,max_seq_length=25,truncate=True,padding=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
t1=get_image_transforms()
p='/content/drive/MyDrive/MAMI_DATASET/newtrain/TRAINING'
b=ViTDataset(traindf,p,tokenizer,t1,25)

train_size = int(0.9 * len(b))
test_size = len(b) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(b, [train_size, test_size])

In [ ]:
train_sampler = RandomSampler(train_dataset)
train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        sampler=train_sampler,
        batch_size= 8,
        collate_fn=collate_fn,
    )
testset_sampler = RandomSampler(test_dataset)
test_dataloader = torch.utils.data.DataLoader(
        test_dataset,
        sampler=testset_sampler,
        batch_size= 8,
        collate_fn=collate_fn,
    )

In [ ]:
class modified(nn.Module):
    def __init__(self):
        super(modified, self).__init__()
        self.bert_model = bertmodel
        self.image_model = vitmodel
        self.pre_classifier = nn.Linear(768*2,768*2)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768*2, 1)
        self.relu = nn.ReLU()
      
        
    def forward(self,ids,mask,img_f):
        bert_out= self.bert_model(ids,attention_mask=mask,return_dict=False)
        x= bert_out[0]
        x = x[:,0]
        vit_out = self.image_model(img_f)
        vout= vit_out[0]
        vf = vout[:,0]
        full_layer=torch.cat((x,vf),dim=1)

        pooler = self.pre_classifier(full_layer)
        outf= self.relu(pooler)
        outf = self.dropout(outf)
        output = self.classifier(outf)
        return output

model= modified()

In [ ]:
def save_checkpoint(save_path, model, valid_loss):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                  'valid_loss': valid_loss}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')
    
def load_checkpoint(load_path, model):
    
    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])
    return model

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

modified(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [ ]:
import torch.optim as optim

In [ ]:
# Creating the loss function and optimizer
num_train_epochs=3
adam_epsilon = 1e-8
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=2e-5,eps=adam_epsilon)

In [ ]:
def evaluate(model, tokenizer, criterion, dataloader, tres = 0.5): 
    
    # Eval!
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    proba = None
    out_label_ids = None
    for batch in dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            ids = batch[0].to(device)
            masked = batch[1].to(device)
            feature = batch[2].to(device)
            labels = batch[3].to(device)
            labels=labels.view(labels.size(0),-1)
            labels=labels.to(torch.float32)
            logits = model(ids,mask=masked,img_f=feature)
            tmp_eval_loss = criterion(logits, labels) 
            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if preds is None:
            preds = torch.sigmoid(logits).detach().cpu().numpy() > tres
            proba = torch.sigmoid(logits).detach().cpu().numpy()
            out_label_ids = labels.detach().cpu().numpy()
        else:            
            preds = np.append(preds, torch.sigmoid(logits).detach().cpu().numpy() > tres, axis=0)
            proba = np.append(proba, torch.sigmoid(logits).detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, labels.detach().cpu().numpy(), axis=0)
    
    eval_loss = eval_loss / nb_eval_steps

    result = {
        "loss": eval_loss,
        "accuracy": accuracy_score(out_label_ids, preds),
        # "AUC": roc_auc_score(out_label_ids, proba),
        "micro_f1": f1_score(out_label_ids, preds, average="micro"),
        "prediction": preds,
        "labels": out_label_ids,
        "proba": proba
    }
    
    return result

In [ ]:
%cd '/content/drive/MyDrive/MAMI_DATASET/'

/content/drive/.shortcut-targets-by-id/17XscmxsoXFqDaSoUSgBQOqCAyuke8ZIC/MAMI_DATASET


In [ ]:
optimizer_step = 0
global_step = 0
train_step = 0
tr_loss, logging_loss = 0.0, 0.0
best_valid_f1 = 0.75
global_steps_list = []
train_loss_list = []
val_loss_list = []
val_acc_list = []
val_auc_list = []
eval_every = len(train_dataloader) // 7
running_loss = 0
file_path='/content/drive/MyDrive/MAMI_DATASET/hybrid'

model.zero_grad()

for i in range(num_train_epochs):
    print("Epoch", i+1, f"from {num_train_epochs}")
    whole_y_pred=np.array([])
    whole_y_t=np.array([])
    for step, batch in enumerate(tqdm(train_dataloader)):
        model.train()
        ids = batch[0].to(device)
        masked = batch[1].to(device)
        feature = batch[2].to(device)
        labels = batch[3].to(device)
        labels=labels.view(labels.size(0),-1)
        labels=labels.to(torch.float32)
        outputs = model(ids,mask=masked,img_f=feature)
        #print(outputs)
        logits = outputs# model outputs are always tuple in transformers (see doc)
        loss = criterion(logits, labels) 
        
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

                        
        if (step + 1) % eval_every == 0:
            
            average_train_loss = running_loss / eval_every
            train_loss_list.append(average_train_loss)
            global_steps_list.append(global_step)
            running_loss = 0.0  
            
            val_result = evaluate(model, tokenizer, criterion, test_dataloader)
            
            val_loss_list.append(val_result['loss'])
            val_acc_list.append(val_result['accuracy'])
            # val_auc_list.append(val_result['AUC'])
            val_auc_list.append(val_result['micro_f1'])
            
            # checkpoint
            if val_result['micro_f1'] > best_valid_f1:
                # best_valid_auc = val_result['AUC']
                val_loss = val_result['loss']
                val_acc = val_result['accuracy']
                best_valid_f1=val_result['micro_f1']
                model_path = f'{file_path}/f1{best_valid_f1:.3f}-loss{val_loss:.3f}-acc{val_acc:.3f}.pt'
                print(f"AUC improved, so saving this model")  
                save_checkpoint(model_path, model, val_result['loss'])              
            
            print("Train loss:", f"{average_train_loss:.4f}"
                  "Val loss:", f"{val_result['loss']:.4f}",
                  "Val acc:", f"{val_result['accuracy']:.4f}",
                  "Val f1:", f"{val_result['micro_f1']:.4f}")
                #   "AUC:", f"{val_result['AUC']:.4f}")   
    print('\n')

Epoch 1 from 3


 14%|█▍        | 159/1125 [14:53<1:22:50,  5.15s/it]

AUC improved, so saving this model


 14%|█▍        | 160/1125 [26:19<56:06:49, 209.34s/it]

Model saved to ==> /content/drive/MyDrive/MAMI_DATASET/hybrid/f10.780-loss0.475-acc0.780.pt
Train loss: 0.0000Val loss: 0.4750 Val acc: 0.7800 Val f1: 0.7800


...

 85%|████████▌ | 960/1125 [07:18<18:48,  6.84s/it]

Train loss: 0.0000Val loss: 0.4552 Val acc: 0.8310 Val f1: 0.8310


100%|█████████▉| 1120/1125 [08:32<00:32,  6.58s/it]

Train loss: 0.0000Val loss: 0.5154 Val acc: 0.8410 Val f1: 0.8410


100%|██████████| 1125/1125 [08:34<00:00,  2.19it/s]

In [ ]:
path='/content/drive/MyDrive/MAMI_DATASET/hybrid/f10.857-loss0.337-acc0.857.pt'
bmodel=load_checkpoint(path,model)

Model loaded from <== /content/drive/MyDrive/MAMI_DATASET/hybrid/f10.857-loss0.337-acc0.857.pt


In [ ]:
class testViTDataset(Dataset):
    def __init__(self, dataset,path, tokenizer, transforms, max_seq_length):
        self.image_ids = dataset.file_name
        self.img_dir = path
        self.tokenizer = tokenizer
        self.transforms = transforms
        self.max_seq_length = max_seq_length
        self.text = dataset.text

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self,idx):
        sentence = torch.LongTensor(self.tokenizer.encode(self.text[idx], add_special_tokens=True))
        # image path
        img_path =os.path.join(self.img_dir,self.image_ids[idx])
        image = Image.open(img_path).convert("RGB")
        # transform image
        if self.transforms:
                image = self.transforms(image)

        res ={
            "sentence":sentence,
            "image": image}
        
        return res

In [ ]:
def test_collate_fn(batch):
    lens = [len(row["sentence"]) for row in batch]
    bsz, max_seq_len = len(batch), max(lens)

    mask_tensor = torch.zeros(bsz, max_seq_len, dtype=torch.long)
    text_tensor = torch.zeros(bsz, max_seq_len, dtype=torch.long)

    for i_batch, (input_row, length) in enumerate(zip(batch, lens)):
        text_tensor[i_batch, :length] = input_row["sentence"]
        mask_tensor[i_batch, :length] = 1

    img_tensor = torch.stack([row["image"] for row in batch])
    
    return text_tensor, mask_tensor, img_tensor

In [ ]:
testdf=pd.read_csv('/content/drive/MyDrive/MAMI_DATASET/Eval/test/Test.csv', sep='\t')
testdf.columns = [*testdf.columns[:-1], 'text']
testdf['text'] =testdf['text'].apply(str).apply(lambda x: simple_text_clean(x))
testdf

,file_name,text
0,15236.jpg,facebook singles groups belike when a new woma...
1,15805.jpg,"so, if you're a feminist how can you eat dairy?"
2,16254.jpg,when a cute girl left your message on seen
3,16191.jpg,photographing something you want to show every...
4,15952.jpg,hey babe can you make me a sandwich? hey babe ...
...,...,...
995,15591.jpg,it's not your fault you didn't design the dres...
996,15049.jpg,think about how much better her skin is breath...
997,15363.jpg,the stereotypes are true f she does have a tig...
998,15199.jpg,draws naked pictures of black women gets mad...


In [ ]:
testp='/content/drive/MyDrive/MAMI_DATASET/Eval/test'
eval_dataset=testViTDataset(testdf,testp,tokenizer,t1,25)
eval_sampler = SequentialSampler(eval_dataset)
eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=1, collate_fn=test_collate_fn )

In [ ]:
def final_prediction(model, dataloader): 
        preds = None
        proba = None
        all_ids = None
        for batch in tqdm(dataloader):
            model.eval()
            batch = tuple(t.to(device) for t in batch)
            with torch.no_grad():
                # ids = batch[5]
                ids = batch[0].to(device)
                masked = batch[1].to(device)
                feature = batch[2].to(device)
                outputs = model(ids,mask=masked,img_f=feature)
                logits = outputs
            if preds is None:
                # all_ids = ids.detach().cpu().numpy()
                preds = torch.sigmoid(logits).detach().cpu().numpy() > 0.5
                proba = torch.sigmoid(logits).detach().cpu().numpy()            
            else:  
                # all_ids = np.append(all_ids, ids.detach().cpu().numpy(), axis=0)
                preds = np.append(preds, torch.sigmoid(logits).detach().cpu().numpy() > 0.5, axis=0)
                proba = np.append(proba, torch.sigmoid(logits).detach().cpu().numpy(), axis=0)

        result = {
            # "ids": all_ids,
            "preds": preds,
            "probs": proba,
        }

        return result

In [ ]:
results_final=final_prediction(bmodel, eval_dataloader)

100%|██████████| 1000/1000 [12:19<00:00,  1.35it/s]


In [ ]:
resf=pd.DataFrame(results_final['preds'])
resf.columns=['label']
resf.loc[resf['label']== True, "label"] = 1
resf.loc[resf['label']== False, "label"] = 0
resf['label']=resf['label'].astype(str).astype(int)

In [ ]:
actualLabel = pd.read_csv("/content/drive/MyDrive/MAMI_DATASET/test/test_labels.txt", sep="\t", header=None, 
                 names=["id", "a","b","c","d","e"])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(actualLabel.a, resf.label))

              precision    recall  f1-score   support

           0       0.80      0.55      0.65       500
           1       0.66      0.86      0.75       500

    accuracy                           0.71      1000
   macro avg       0.73      0.71      0.70      1000
weighted avg       0.73      0.71      0.70      1000



In [ ]:
f1_score(actualLabel.a, resf.label,average='macro')

0.7007166401548074